Project for DISCERN 2024
===

This project is supposed to use [gpt2 model from huggingface](https://huggingface.co/openai-community/gpt2) and then use train data from [kaggle](https://www.kaggle.com/datasets/emineyetm/fake-news-detection-datasets) to detect fake and true information in news and articles. The articles used for checking collected by our team [are here](https://unirau-my.sharepoint.com/:x:/g/personal/dovhan_o_nikita22_stud_rau_ro/EVZaoVJ1OIFFmkT7ognXzbcBiR8JDDXK-ID0DWAdiFnMvg?e=DTESAz).

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, TFGPT2Model, TFGPT2ForSequenceClassification
import tensorflow as tf

Parameters
---

In [ ]:
true_file = "data/True.csv"
fake_file = "data/Fake.csv"

model_name = "gpt2"
model_dir = "results/2.model"

amount_of_entries = 1500
batch_size = 10

In [ ]:
true_df = pd.read_csv(true_file)
fake_df = pd.read_csv(fake_file)

In [ ]:
true_df["label"] = 1
fake_df["label"] = 0

In [ ]:
combined_df = pd.concat([true_df, fake_df], ignore_index=True)
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)
combined_df = combined_df.head(amount_of_entries)

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    combined_df["text"].values,
    combined_df["label"].values,
    test_size=0.2,
    random_state=42
)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512, return_tensors='tf')
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=512, return_tensors='tf')

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))


In [ ]:
class GPT2Classifier(tf.keras.Model):
    def __init__(self, num_classes):
        super(GPT2Classifier, self).__init__()
        self.gpt2 = TFGPT2Model.from_pretrained(model_name)
        self.dropout = tf.keras.layers.Dropout(0.6)
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        outputs = self.gpt2(inputs)[0]
        pooled_output = tf.reduce_mean(outputs, axis=1)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

model = GPT2Classifier(num_classes=2)
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy() 
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
model.fit(train_dataset.shuffle(1000).batch(batch_size), epochs=2, batch_size=batch_size)

In [ ]:
loss, accuracy = model.evaluate(test_dataset.batch(32))
print(f"Test accuracy: {accuracy}")

In [ ]:
model.save(model_dir)